<a href="https://colab.research.google.com/github/RishabhGithub7348/YouTube-Video-Summarizer-and-Quiz-Generator/blob/main/YouTube_Video_Summarizer_and_Quiz_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!chmod 400 ./monkey-science-gpu.pem
!ssh -o "StrictHostKeyChecking=no" -i "monkey-science-gpu.pem" ubuntu@ec2-13-235-135-35.ap-south-1.compute.amazonaws.com

In [ ]:
!pip install youtube_transcript_api transformers torch nltk


# YouTube Video Summarizer and Quiz Generator

This project creates a summary and a quiz based on the transcript of a YouTube video.

## Setup

First, let's install the required libraries:


In [ ]:
!pip install youtube_transcript_api transformers torch nltk

Now, let's import the necessary libraries and download the required NLTK data:

In [ ]:
import torch
import random
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import pipeline
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tag import pos_tag

## Transcript Retrieval

This function retrieves the transcript of a YouTube video, prioritizing English subtitles:

In [ ]:
def get_transcript(video_url):
    """
    Retrieves the transcript of a YouTube video.

    Args:
    video_url (str): The URL of the YouTube video.

    Returns:
    str: The transcript text if available, None otherwise.
    """
    try:
        video_id = video_url.split("v=")[1]
        transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

        # Try to get English transcript
        try:
            transcript = transcript_list.find_transcript(['en'])
        except:
            # If English is not available, get the first available transcript
            try:
                transcript = transcript_list.find_transcript(['en-US', 'en-GB'])
            except:
                # If still no transcript, get any available transcript
                transcript = transcript_list[0]

        return " ".join([entry['text'] for entry in transcript.fetch()])
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

## Text Summarization

This function uses a pre-trained T5 model to summarize the transcript:

In [ ]:
def summarize_text(text, max_length=150):
    """
    Summarizes the given text using a T5 model.

    Args:
    text (str): The text to summarize.
    max_length (int): The maximum length of the summary.

    Returns:
    str: The summarized text.
    """
    summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="pt")
    summary = summarizer(text, max_length=max_length, min_length=30, do_sample=False)
    return summary[0]['summary_text']

## Multiple Choice Question Generation

This function generates multiple-choice questions based on the summary:

In [ ]:
def generate_mcq(text, num_questions=3):
    """
    Generates multiple-choice questions from the given text.

    Args:
    text (str): The text to generate questions from.
    num_questions (int): The number of questions to generate.

    Returns:
    list: A list of dictionaries containing questions, options, and correct answers.
    """
    sentences = sent_tokenize(text)
    questions = []

    for sentence in sentences[:num_questions]:
        words = word_tokenize(sentence)
        tagged = pos_tag(words)

        nouns = [word for word, pos in tagged if pos.startswith('NN')]
        if not nouns:
            continue

        blank_word = random.choice(nouns)
        question = sentence.replace(blank_word, "________")

        options = [blank_word]
        other_nouns = [word for word in nouns if word != blank_word]
        options.extend(random.sample(other_nouns, min(2, len(other_nouns))))

        while len(options) < 4:
            random_word = random.choice([word for word, pos in tagged if pos.startswith('N') or pos.startswith('V') or pos.startswith('JJ')])
            if random_word not in options:
                options.append(random_word)

        random.shuffle(options)

        questions.append({
            'question': question,
            'options': options,
            'correct_answer': blank_word
        })

    return questions

## Video Processing and Quiz

This function processes the video, generates a summary and quiz, and calculates the score:

In [ ]:
def process_video(video_url, num_questions):
    """
    Processes a YouTube video: retrieves transcript, generates summary and quiz.

    Args:
    video_url (str): The URL of the YouTube video.
    num_questions (int): The number of questions to generate.
    """
    transcript = get_transcript(video_url)

    while not transcript:
        print("This video doesn't have subtitles or the subtitles couldn't be retrieved.")
        video_url = input("Please enter a different YouTube video URL with subtitles: ")
        transcript = get_transcript(video_url)

    if transcript:
        summary = summarize_text(transcript)
        mcqs = generate_mcq(summary, num_questions)

        print("\nSummary:")
        print(summary)
        print("\nMultiple Choice Questions:")

        correct_answers = 0
        for i, mcq in enumerate(mcqs, 1):
            print(f"\n{i}. {mcq['question']}")
            for j, option in enumerate(mcq['options'], 1):
                print(f"   {j}. {option}")

            while True:
                user_answer = input("Enter your answer (1-4): ")
                if user_answer in ['1', '2', '3', '4']:
                    user_answer = int(user_answer)
                    if mcq['options'][user_answer - 1] == mcq['correct_answer']:
                        print("Correct!")
                        correct_answers += 1
                    else:
                        print(f"Wrong. The correct answer is: {mcq['correct_answer']}")
                    break
                else:
                    print("Invalid input. Please enter a number between 1 and 4.")

        score_percentage = (correct_answers / num_questions) * 100
        print(f"\nYour score: {correct_answers}/{num_questions}")
        print(f"Percentage: {score_percentage:.2f}%")
    else:
        print("Failed to process the video.")

## Main Function

This is the main function that runs the program:

In [ ]:
def main():
    """
    Main function to run the program.
    """
    video_url = input("Enter a YouTube video URL: ")
    num_questions = 3  # Fixed number of questions

    process_video(video_url, num_questions)

if __name__ == "__main__":
    main()

## Run the Program

To run the program, execute the following cell:

In [ ]:
main()

Enter a YouTube video URL: https://www.youtube.com/watch?v=B6mi1-YoRT4

Summary:
force is a push or pull upon an object, resulting from its interaction with another object . there are forces acting on a ball when it is stationary, not moving at all . if there is no force acting on the ball, it will continue to move at the same velocity .

Multiple Choice Questions:

1. force is a push or pull upon an object, resulting from its ________ with another object .
   1. push
   2. is
   3. interaction
   4. object
Enter your answer (1-4): 3
Correct!

2. there are forces acting on a ________ when it is stationary, not moving at all .
   1. ball
   2. forces
   3. is
   4. acting
Enter your answer (1-4): 1
Correct!

3. if there is no ________ acting on the ball, it will continue to move at the same velocity .
   1. force
   2. acting
   3. velocity
   4. ball
Enter your answer (1-4): 1
Correct!

Your score: 3/3
Percentage: 100.00%


This program will:
1. Ask for a YouTube video URL
2. Retrieve the video's transcript
3. Generate a summary of the transcript
4. Create 3 multiple-choice questions based on the summary
5. Allow the user to answer these questions
6. Provide a score at the end

Note: Make sure the YouTube video you choose has subtitles available for the best results.